### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happenned of Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 24 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)



In [157]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

filings_dfs = pd.read_html(response.text)

C:\Users\rcata\AppData\Local\Temp\ipykernel_17740\503092549.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  filings_dfs = pd.read_html(response.text)


In [158]:
filings = filings_dfs[0]
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     329 non-null    object
 1   Symbol          329 non-null    object
 2   Company Name    329 non-null    object
 3   Price Range     329 non-null    object
 4   Shares Offered  329 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


In [159]:
filings['Filing Date'] = pd.to_datetime(filings['Filing Date'])

In [160]:
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     329 non-null    datetime64[ns]
 1   Symbol          329 non-null    object        
 2   Company Name    329 non-null    object        
 3   Price Range     329 non-null    object        
 4   Shares Offered  329 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 13.0+ KB


In [161]:
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'].str.replace('-', ''), errors='coerce')

In [162]:
filings['Shares Offered'] = pd.to_numeric(filings['Shares Offered'])

In [163]:
filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     329 non-null    datetime64[ns]
 1   Symbol          329 non-null    object        
 2   Company Name    329 non-null    object        
 3   Price Range     329 non-null    object        
 4   Shares Offered  253 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 13.0+ KB


In [164]:
filings['Price Range'].unique()

array(['$10.00', '-', '$5.00', '$4.00 - $5.00', '$4.00 - $4.50',
       '$5.00 - $6.00', '$4.00 - $6.00', '$5.00 - $7.00',
       '$10.00 - $12.00', '$3.00 - $4.00', '$4.00', '$4.13',
       '$10.00 - $14.00', '$3.50 - $4.50', '$7.00 - $8.00',
       '$8.00 - $10.00', '$11.25 - $13.75', '$4.00 - $4.75', '$4.30',
       '$9.00 - $11.00', '$3.00 - $5.00', '$4.50 - $5.50',
       '$5.75 - $6.75', '$8.00 - $9.00', '$5.50', '$4.35 - $6.35',
       '$20.00', '$6.00 - $6.50', '$6.00 - $7.00', '$6.00', '$4.45',
       '$8.00', '$15.00', '$6.25', '$4.25 - $6.25', '$5.20 - $7.20',
       '$4.50 - $6.50', '$4.25', '$7.00 - $9.00', '$5.75',
       '$18.00 - $20.00', '$7.00 - $7.50', '$8.50 - $9.50',
       '$5.00 - $6.50'], dtype=object)

In [165]:
import re
import numpy as np

def calculate_avg_price(price_range):
    # Regular expression pattern to extract numbers from the price range string
    pattern = r'\d+\.\d+?'
    
    # Find all numbers in the price range string
    prices = re.findall(pattern, price_range)
    print(prices)
    # Convert numbers to float and handle cases where only one price is provided or no price is specified
    if len(prices) == 0:
        return np.nan
    elif len(prices) == 1:
        return float(prices[0])
    else:
        # Calculate the average of two prices if a range is given
        return (float(prices[0]) + float(prices[1])) / 2

# Example usage:
price_range = "10.5 - 15.5"
avg_price = calculate_avg_price(price_range)
print("Average Price:", avg_price)


['10.5', '15.5']
Average Price: 13.0


In [166]:
filings['Avg_price'] = filings['Price Range'].apply(calculate_avg_price)

['10.0']
[]
['5.0']
['10.0']
[]
['5.0']
[]
[]
[]
[]
['4.0', '5.0']
['4.0', '4.5']
['4.0', '5.0']
[]
[]
['4.0', '5.0']
['5.0', '6.0']
[]
['10.0']
[]
['4.0', '6.0']
['5.0', '7.0']
[]
['4.0', '6.0']
['10.0']
['10.0', '12.0']
[]
['4.0', '6.0']
[]
['3.0', '4.0']
[]
['4.0', '5.0']
['4.0']
['4.0', '5.0']
['4.0', '5.0']
['4.1']
[]
[]
['5.0']
['4.0', '5.0']
[]
['4.0', '6.0']
['4.0', '5.0']
['5.0', '6.0']
[]
['10.0', '14.0']
['4.0', '6.0']
[]
['4.0', '6.0']
[]
[]
['10.0']
['3.5', '4.5']
[]
['7.0', '8.0']
['4.0', '5.0']
[]
[]
['4.0', '6.0']
['8.0', '10.0']
[]
['11.2', '13.7']
[]
[]
['4.0', '6.0']
['4.0', '6.0']
[]
['10.0']
[]
['4.0']
['5.0']
['4.0', '5.0']
['4.0', '5.0']
[]
[]
['4.0']
['10.0']
['4.0', '4.7']
['5.0']
['3.5', '4.5']
['8.0', '10.0']
['4.0', '5.0']
['4.0', '6.0']
['4.3']
['9.0', '11.0']
[]
['4.0', '5.0']
['5.0', '7.0']
['4.0', '6.0']
['10.0']
['3.0', '5.0']
['4.5', '5.5']
[]
['4.0']
['4.0', '5.0']
['5.7', '6.7']
['4.0']
['5.0', '7.0']
[]
[]
['4.0']
['5.0', '6.0']
['4.0', '6.0']
['8.0

In [167]:
filings['Shares_offered_value'] = filings.apply(lambda row: row['Shares Offered'] * row['Avg_price'] if row['Shares Offered'] != np.nan and row['Avg_price'] != np.nan else np.nan, axis=1)

In [168]:
# Combine all DataFrames in ipo_dfs into a single DataFrame
combined_df = pd.concat([filings], ignore_index=True)

# Filter records for 2023 and Fridays
fridays_2023_df = combined_df[(combined_df['Filing Date'].dt.year == 2023) & (combined_df['Filing Date'].dt.dayofweek == 4)]
# Calculate the total sum in millions of USD
total_sum_millions_usd = fridays_2023_df['Shares_offered_value'].sum()/1e6  # Convert to millions of USD

# Round the total sum to the closest integer
total_sum_millions_usd_rounded = round(total_sum_millions_usd)

print("Total sum in millions of USD for filings on Fridays in 2023:", total_sum_millions_usd_rounded)


Total sum in millions of USD for filings on Fridays in 2023: 276


### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). 
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance. 

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks.
Some of the tickers like 'DYCQ' and 'LEGT' were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Addtional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


---

### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS.

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

---

### Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.
You need to sum these results over all trades (460 times in 10 years).

Additional:
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
  * are you still profitable on those trades?


---

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

---
## Submitting the solutions

Form for submitting: https://courses.datatalks.club/sma-zoomcamp-2024/homework/hw02

-